In [ ]:
import shutil
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os

desktop_path = os.path.expanduser("~/Desktop")  # Gets the desktop path
directory = os.path.join(desktop_path, "Plantas", "PlantVillage")

Cargar y preprocesar las imágenes

In [ ]:
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,          # Normalización [0, 1]
    rotation_range=40,       # Rotación aleatoria
    width_shift_range=0.2,   # Desplazamiento horizontal
    height_shift_range=0.2,  # Desplazamiento vertical
    shear_range=0.2,         # Deformación
    zoom_range=0.2,          # Zoom aleatorio
    horizontal_flip=True,    # Volteo horizontal
    fill_mode='nearest' 
)

val_datagen = ImageDataGenerator(rescale=1./255)

# Generadores de datos
batch_size = 32
img_size = (224, 224)  # ResNet50 espera 224x224 por defecto

train_generator = train_datagen.flow_from_directory(
    directory=directory,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    directory=directory,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


Verificar las clases cargadas

In [ ]:
print("Clases encontradas:", train_generator.class_indices)
print("Número de clases:", train_generator.num_classes)


Visualizar imágenes del batch

In [ ]:
x_batch, y_batch = next(train_generator)

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(x_batch[i])
    plt.title(np.argmax(y_batch[i]))
    plt.axis('off')
plt.show()



Código base: loop de experimentos entrenamos 5 épocas en feature‑extraction (capas congeladas), guardamos el mejor val_accuracy, liberamos memoria y seguimos. Más adelante afinamos el ganador.

In [ ]:
import tensorflow as tf, gc, pandas as pd, time
from tensorflow.keras import layers, models
from tensorflow.keras.applications import (
    MobileNetV2, EfficientNetB0, ResNet50, InceptionV3, DenseNet121
)

IMG_SIZE = (224, 224); BATCH = 32; EPOCHS = 5
BACKBONES = {
    "MobileNetV2": MobileNetV2,
    "EfficientNetB0": EfficientNetB0,
    "ResNet50": ResNet50,
    "InceptionV3": InceptionV3,
    "DenseNet121": DenseNet121
}

results = []

for name, constructor in BACKBONES.items():
    print(f"\n🔄 Entrenando {name}")
    base = constructor(weights="imagenet", include_top=False,
                       input_shape=IMG_SIZE + (3,))
    base.trainable = False
    
    x = layers.GlobalAveragePooling2D()(base.output)
    x = layers.Dropout(0.3)(x)
    out = layers.Dense(train_generator.num_classes, activation="softmax")(x)
    model = models.Model(base.input, out)
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    
    t0 = time.time()
    hist = model.fit(
        train_generator,
        validation_data=val_generator,
        epochs=EPOCHS,
        verbose=1
    )
    t_train = time.time() - t0
    
    best_val = max(hist.history["val_accuracy"])
    results.append({"model": name, "val_acc": best_val, "train_time_s": t_train})
    
    # liberar memoria GPU/CPU
    del model, base, hist; gc.collect(); tf.keras.backend.clear_session()

pd.DataFrame(results).sort_values("val_acc", ascending=False)



Usar EarlyStopping(patience=2, restore_best_weights=True) para evitar sobreajuste incluso en estas 5 épocas.

Se puede reducir IMG_SIZE a 160 × 160 para acelerar las redes grandes en Colab Free.

Elegir el mejor modelo y fine‑tuning

In [ ]:
best_name = "MobileNetV2"

base = MobileNetV2(weights="imagenet", include_top=False,
                      input_shape=IMG_SIZE + (3,))
# 1) Feature extraction head de nuevo
...
model = ...
# 2) Cargar pesos previos si los guardaste (opcional)
# model.load_weights("efficientnet_b0_feature_ext.h5")

# 3) Descongelar últimas N capas
for layer in base.layers[-30:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # lr bajo para fine‑tune
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

ft_hist = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    callbacks=[EarlyStopping(patience=3, restore_best_weights=True)]
)

model.save("MobileNetV2_finetuned.h5")



Evaluación final y matriz de confusión

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns; import matplotlib.pyplot as plt
import numpy as np

# Generador de test (sin augmentations)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(
    directory=directory,
    subset=None,  # si lo separaste en /test/
    shuffle=False,
    target_size=IMG_SIZE,
    batch_size=BATCH,
    class_mode='categorical'
)

y_true = test_gen.classes
y_pred = model.predict(test_gen, verbose=1)
y_pred_labels = np.argmax(y_pred, axis=1)

print(classification_report(y_true, y_pred_labels, target_names=test_gen.class_indices.keys()))

cm = confusion_matrix(y_true, y_pred_labels)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt="d", xticklabels=test_gen.class_indices.keys(),
            yticklabels=test_gen.class_indices.keys())
plt.xlabel("Predicted"); plt.ylabel("True"); plt.title("Confusion Matrix")
plt.show()

Conversión básica a TFLite (para modelos basados en TensorFlow)

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer

# Cargar modelo y tokenizador (ejemplo con T5)
model_name = "MobileNetV2-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

# Guardar modelo en formato SavedModel
model.save_pretrained("./MobileNetV2_saved_model", saved_model=True)

# Convertir a TFLite
converter = tf.lite.TFLiteConverter.from_saved_model("./MobileNetV2_saved_model/saved_model/1")
tflite_model = converter.convert()

# Guardar modelo TFLite
with open("MobileNetV2_model.tflite", "wb") as f:
    f.write(tflite_model)

Exportar a TensorFlow Lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("plant_disease.tflite", "wb").write(tflite_model)

Conversión avanzada con cuantización (para reducir tamaño)

In [ ]:
# Configurar conversor con cuantización
converter = tf.lite.TFLiteConverter.from_saved_model("./MobileNetV2_saved_model/saved_model/1")

# Cuantización dinámica (balance entre tamaño/calidad)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Opcional: Especificar representantes para cuantización completa
def representative_dataset():
    for _ in range(100):
        yield [tf.random.uniform(shape=(1, 128), dtype=tf.int32)]

converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

tflite_quant_model = converter.convert()

with open("MobileNetV2_model_quant.tflite", "wb") as f:
    f.write(tflite_quant_model)

Alternativas recomendadas Para tu aplicación de portfolio, considera:
Opción A: Usar Hugging Face Pipelines con API

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model='MobileNetV2', device=0)

Opción B: Optimización con ONNX Runtime (mejor soporte para NLP)

In [ ]:
#pip install optimum[onnxruntime]

from optimum.onnxruntime import ORTModelForSeq2SeqLM

model = ORTModelForSeq2SeqLM.from_pretrained("MobileNetV2-small", from_transformers=True)